In [21]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sns
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split

* Filename - уникальный id испытуемого.
* Test_index - номер группы, нумерация с 0
* Presentation - номер повторения, нумерация с 1
* Question - номер вопроса, нумерация с 0
* Data - равномерно зарегистрированные полиграфом 12 секунд сигнала с частотой 20 Гц (240 значений) (метка датчика регистрации сигнала - фотоплетизмограмма)
* Data_2 - равномерно зарегистрированные полиграфом 12 секунд сигнала с частотой 20 Гц (240 значений) (метка датчика регистрации сигнала - пьезоплетизмограмма)
* Class_label  Data_type - метка класса (0 - слабый стресс, 1 - средний стресс, 2 - сильный стресс)


In [2]:
df = pd.read_excel('data/dataset_train.xlsx')

In [15]:
df.groupby(['Filename', 'Class_label']).count()

Test_index  Presentation  \
Filename                             Class_label                             
00b38188-82ee-4f1d-9661-485338815751 0                     4             4   
                                     1                    46            46   
                                     2                    27            27   
02a1eb88-b6b1-49db-975d-ea2c0edfdd43 0                    19            19   
                                     1                    66            66   
...                                                      ...           ...   
fefd767f-e663-4b87-b9ee-314cf463ce7d 1                    37            37   
                                     2                    17            17   
ff9f1820-4dbc-47fb-ac2f-eeff31cb1501 0                    20            20   
                                     1                    51            51   
                                     2                    13            13   

                                                  Question  Data  Data_2  
Filename                             Class_label                          
00b38188-82ee-4f1d-9661-485338815751 0                   4     4       4  
                                     1                  46    46      46  
                                     2                  27    27      27  
02a1eb88-b6b1-49db-975d-ea2c0edfdd43 0                  19    19      19  
                                     1                  66    66      66  
...                                                    ...   ...     ...  
fefd767f-e663-4b87-b9ee-314cf463ce7d 1                  37    37      37  
                                     2                  17    17      17  
ff9f1820-4dbc-47fb-ac2f-eeff31cb1501 0                  20    20      20  
                                     1                  51    51      51  
                                     2                  13    13      13  

[1159 rows x 5 columns]

In [41]:
from tqdm.auto import tqdm

prepared_df = df[['Test_index', 'Presentation', 'Question','Class_label']].copy()
for stat in tqdm([np.mean, np.std, np.min, np.max]):
    for d in ['Data', 'Data_2']:
        prepared_df[f'{str(stat)}_{d}'] = df[d].apply(lambda x: stat(eval(x)) if len(x) != 2 else 0)
prepared_df.head()

  0%|          | 0/4 [00:00<?, ?it/s]

,Test_index,Presentation,Question,Class_label,<function mean at 0x7f9ed03b4550>_Data,<function mean at 0x7f9ed03b4550>_Data_2,<function std at 0x7f9ed03b4700>_Data,<function std at 0x7f9ed03b4700>_Data_2,<function amin at 0x7f9ed03b5940>_Data,<function amin at 0x7f9ed03b5940>_Data_2,<function amax at 0x7f9ed03b5790>_Data,<function amax at 0x7f9ed03b5790>_Data_2
0,2,3,7,1,-17.716667,-114.825000,304.736120,914.712470,-834,-2038,390,1899
1,0,1,0,0,-120.504167,538.358333,228.359877,39.566568,-638,456,361,630
2,3,3,1,2,18.212500,-162.187500,418.567986,441.237722,-814,-862,687,645
3,0,3,6,1,-136.404167,628.725000,122.910364,854.320787,-437,-1463,163,2047
4,3,1,2,2,1.600000,-842.808333,170.881216,115.305702,-294,-1056,300,-628


In [46]:
train_df, test_df = train_test_split(prepared_df, test_size=0.2, random_state=142, stratify=prepared_df.Class_label)



In [47]:
ctb = CatBoostClassifier(verbose=100)
ctb.fit(train_df.drop('Class_label', axis=1), train_df.Class_label)
y_pred = ctb.predict(test_df.drop('Class_label', axis=1))


Learning rate set to 0.093264
0:	learn: 1.0776289	total: 64.3ms	remaining: 1m 4s
100:	learn: 0.9520323	total: 554ms	remaining: 4.93s
200:	learn: 0.9286666	total: 1.13s	remaining: 4.5s
300:	learn: 0.9080446	total: 1.64s	remaining: 3.8s
400:	learn: 0.8892836	total: 2.11s	remaining: 3.16s
500:	learn: 0.8725278	total: 2.61s	remaining: 2.6s
600:	learn: 0.8564451	total: 3.09s	remaining: 2.05s
700:	learn: 0.8411254	total: 3.57s	remaining: 1.52s
800:	learn: 0.8270280	total: 4.08s	remaining: 1.01s
900:	learn: 0.8127416	total: 4.57s	remaining: 502ms
999:	learn: 0.7988660	total: 5.08s	remaining: 0us


In [49]:
from sklearn.metrics import classification_report

print(classification_report(test_df.Class_label, y_pred))

              precision    recall  f1-score   support

           0       0.38      0.05      0.08      1343
           1       0.56      0.95      0.71      3528
           2       0.36      0.05      0.09      1406

    accuracy                           0.55      6277
   macro avg       0.43      0.35      0.29      6277
weighted avg       0.48      0.55      0.43      6277



In [50]:
import torch
import torch.nn as nn


In [64]:
vals = df.Data.apply(eval)

ValueError: Shape of passed values is (31383, 1), indices imply (31383, 0)

In [73]:
vals = df.Data.apply(eval)
res = [val if len(val) == 240 else val + [0] * (240 - len(val)) for val in vals]
np.array(res)

In [76]:
df_1 = pd.DataFrame(res).add_prefix('data')
df_2 = pd.DataFrame(res).add_prefix('data1')

In [78]:
res_df = pd.concat((df_1, df_2), axis=1)

res_df.isna().sum().max()

0

In [79]:
res_df

,data0,data1,data2,data3,data4,data5,data6,data7,data8,data9,...,data1230,data1231,data1232,data1233,data1234,data1235,data1236,data1237,data1238,data1239
0,298,296,2,-455,-703,-596,-262,-44,28,92,...,-401,-202,-26,58,114,164,220,267,310,310
1,99,75,-228,-588,-583,-465,-394,-363,-339,-257,...,-28,66,204,308,361,294,-17,-249,-337,-294
2,-369,-543,-644,-670,-636,-586,-524,-444,-383,-306,...,31,18,-69,-183,-282,-323,-313,-297,-292,-284
3,-6,-32,-152,-253,-275,-246,-211,-197,-179,-122,...,-44,4,47,61,2,-114,-116,-84,-70,-61
4,165,178,190,202,218,230,200,121,71,81,...,-145,-127,-110,-96,-82,-66,-52,-85,-167,-216
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31378,-35,-20,-3,20,37,53,65,83,93,21,...,61,74,98,125,146,168,201,217,184,115
31379,-550,-540,-510,-450,-350,-262,-161,-59,36,154,...,392,433,368,244,60,-123,-165,-148,-118,-85
31380,98,95,126,206,207,62,-78,-83,-7,132,...,201,229,242,148,14,32,176,218,213,203
31381,-365,-264,-184,-58,102,263,419,598,298,-180,...,-300,-345,-202,-63,35,151,312,456,596,660
